Import SysMLv2 Model:

In [1]:
import syside
import os

MAIN_DIR = "/Users/alejandronietocuatenta/Documents/sysmlv2_omg_introduction/"
# MODEL_DIR = "omg_single_example/"
# MODEL_DIR = "omg_introduction/" # mid complexity example
# MODEL_DIR = "omg_simple_vehicle/" # high complexity example
MODEL_DIR = "project_electronics_basics/"
# MODEL_DIR = "project_clean/"

# Construct full directory path
full_dir = os.path.join(MAIN_DIR, MODEL_DIR)

# Find the first file ending with .sysml
for file in os.listdir(full_dir):
    if file.endswith(".sysml"):
        MODEL_FILE_PATH = os.path.join(full_dir, file)
        break  # stop after finding the first .sysml file

(model, diagnostics) = syside.load_model([MODEL_FILE_PATH])

Check Metamodel structure:

In [ ]:
for doc in model.user_docs:
    print(doc)
    with doc.lock() as locked:
        print(f"Model sexp:\n {syside.sexp(locked.root_node)}")

Check PartDefinition:

In [ ]:
for PartDefinition in model.nodes(syside.PartDefinition):
    print(PartDefinition)
    print(f"\tType::is_abstract: {PartDefinition.is_abstract}")
    print(f"\tOcurrenceDefinition::is_individual: {PartDefinition.is_individual}")
    print(f"\tDefinition::owned_allocations: {len(list(PartDefinition.owned_allocations))}")
    print(f"\tDefinition::owned_parts: {len(list(PartDefinition.owned_parts))}")

Check PartUsage:

In [ ]:
for PartUsage in model.nodes(syside.PartUsage):
    print(PartUsage)
    print(f"\tPartUsage::part_definitions: {len(list(PartUsage.part_definitions))}")
    print(f"\tItemUsage::item_definitions: {len(list(PartUsage.part_definitions))}")
    print(f"\tOccurrenceUsage::individual_definition: {PartUsage.individual_definition}")
    print(f"\tElement::declared_name: {PartUsage.declared_name}")
    print(f"\tElement::documentation: {len(list(PartUsage.documentation))}")
    print(f"\tElement::element_id: {PartUsage.element_id}")
    print(f"\tElement::owner: {PartUsage.owner}")
    print(f"\tElement::owning_membership: {PartUsage.owning_membership}")
    print(f"\tElement::owning_namespace: {PartUsage.owning_namespace}")
None